In [1]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

In [3]:
!pip install pyglet

# **Environment (CartPole-v0) with OpenAI Gym**

In [4]:
import gym
import random

In [5]:
env = gym.make('CartPole-v0')
states = env.observation_space.shape[0]
actions = env.action_space.n

In [6]:
# Cart position
# Cart velocity
# Pole angle
# Pole velocity at tip
print('States:', states)

States: 4


In [7]:
# push cart to the left
# push cart to the right
print('Actions:', actions)

Actions: 2


# **Test Before Use Deep Reinforcement Learning (DRL)**

In [8]:
episodes = 30 # match number
for episode in range(1, episodes+1):
  state = env.reset()
  done = False
  score = 0

  while not done:
    env.render()
    action = random.choice([0,1])
    n_state, reward, done, info = env.step(action)
    score+=reward
  print('Episode:{} Score:{}'.format(episode, score)) 

Episode:1 Score:28.0
Episode:2 Score:17.0
Episode:3 Score:20.0
Episode:4 Score:14.0
Episode:5 Score:15.0
Episode:6 Score:17.0
Episode:7 Score:10.0
Episode:8 Score:29.0
Episode:9 Score:17.0
Episode:10 Score:20.0
Episode:11 Score:19.0
Episode:12 Score:17.0
Episode:13 Score:14.0
Episode:14 Score:12.0
Episode:15 Score:15.0
Episode:16 Score:26.0
Episode:17 Score:41.0
Episode:18 Score:13.0
Episode:19 Score:16.0
Episode:20 Score:11.0
Episode:21 Score:41.0
Episode:22 Score:45.0
Episode:23 Score:28.0
Episode:24 Score:40.0
Episode:25 Score:11.0
Episode:26 Score:15.0
Episode:27 Score:39.0
Episode:28 Score:17.0
Episode:29 Score:16.0
Episode:30 Score:54.0


# **Deep Learning Model (MLP) with Keras**

In [25]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [26]:
# MLP => 2 hidden layers with 24 neurons in each one
#        4 neurons for each state in the input layer
#        2 neurons for each action in the output layer
#        activation function for all the hidden layers = relu 
def build_model(states, actions):
    model = tf.keras.Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [32]:
# MLP structure
model = build_model(states, actions)
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 24)                120       
_________________________________________________________________
dense_7 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_8 (Dense)              (None, 2)                 50        
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


# **Build Agent with Keras-DRL (Deep Reinforcement Learning)**

In [33]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [34]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

# **DRL Training**

In [35]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
    1/10000 [..............................] - ETA: 26:26 - reward: 1.0000

/home/pedro/.local/lib/python3.8/site-packages/rl/memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 129s 13ms/step - reward: 1.0000
109 episodes - episode_reward: 90.092 [8.000, 200.000] - loss: 2.412 - mae: 19.514 - mean_q: 39.526

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 150s 15ms/step - reward: 1.0000
52 episodes - episode_reward: 192.462 [167.000, 200.000] - loss: 3.496 - mae: 40.272 - mean_q: 81.317

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 113s 11ms/step - reward: 1.0000
52 episodes - episode_reward: 195.288 [172.000, 200.000] - loss: 2.528 - mae: 40.471 - mean_q: 81.486

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 113s 11ms/step - reward: 1.0000
50 episodes - episode_reward: 196.780 [166.000, 200.000] - loss: 2.684 - mae: 37.568 - mean_q: 75.457

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 101s 10ms/step - reward: 1.0000
done, took 606.589 seconds


# **DEL Test**

In [36]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 124.000, steps: 124
Episode 2: reward: 114.000, steps: 114
Episode 3: reward: 129.000, steps: 129
Episode 4: reward: 123.000, steps: 123
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 147.000, steps: 147
Episode 7: reward: 150.000, steps: 150
Episode 8: reward: 138.000, steps: 138
Episode 9: reward: 109.000, steps: 109
Episode 10: reward: 122.000, steps: 122
Episode 11: reward: 135.000, steps: 135
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 132.000, steps: 132
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 168.000, steps: 168
Episode 16: reward: 151.000, steps: 151
Episode 17: reward: 111.000, steps: 111
Episode 18: reward: 154.000, steps: 154
Episode 19: reward: 127.000, steps: 127
Episode 20: reward: 114.000, steps: 114
Episode 21: reward: 119.000, steps: 119
Episode 22: reward: 174.000, steps: 174
Episode 23: reward: 185.000, steps: 185
Episode 24: reward: 158.000, steps: 158
Episode 25: reward: 

In [37]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 110.000, steps: 110
Episode 2: reward: 139.000, steps: 139
Episode 3: reward: 134.000, steps: 134
Episode 4: reward: 115.000, steps: 115
Episode 5: reward: 200.000, steps: 200
Episode 6: reward: 156.000, steps: 156
Episode 7: reward: 200.000, steps: 200
Episode 8: reward: 177.000, steps: 177
Episode 9: reward: 132.000, steps: 132
Episode 10: reward: 132.000, steps: 132
Episode 11: reward: 106.000, steps: 106
Episode 12: reward: 124.000, steps: 124
Episode 13: reward: 200.000, steps: 200
Episode 14: reward: 110.000, steps: 110
Episode 15: reward: 200.000, steps: 200


# **Reloading Agent from Memory (Saving Weights)**

In [38]:
dqn.save_weights('dqn_weights.h5f', overwrite=True)

In [39]:
del model
del dqn
del env

In [40]:
env = gym.make('CartPole-v0')
actions = env.action_space.n
states = env.observation_space.shape[0]
model = build_model(states, actions)
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])

In [41]:
dqn.load_weights('dqn_weights.h5f')

In [42]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...
Episode 1: reward: 121.000, steps: 121
Episode 2: reward: 174.000, steps: 174
Episode 3: reward: 145.000, steps: 145
Episode 4: reward: 142.000, steps: 142
Episode 5: reward: 135.000, steps: 135
Episode 6: reward: 152.000, steps: 152
Episode 7: reward: 142.000, steps: 142
Episode 8: reward: 190.000, steps: 190
Episode 9: reward: 143.000, steps: 143
Episode 10: reward: 194.000, steps: 194
Episode 11: reward: 161.000, steps: 161
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 153.000, steps: 153
Episode 14: reward: 110.000, steps: 110
Episode 15: reward: 194.000, steps: 194
